# 6. Tuning

In [1]:
MODEL = 'HGBR'
TUNING_MODEL = 'guanganb'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
import os
import pickle
py_file_location = "./drive/My Drive/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

py_file_location = "./drive/My Drive/LAB/BrownlowH2H/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

import NingXiang

import pandas as pd

In [4]:
ningxiang = NingXiang.NingXiang()

NingXiang Initialised


Read in and Prepare Data

In [5]:
train_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/train_data.parquet')
val_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/val_data.parquet')
test_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/test_data.parquet')

In [6]:
train_other_info = train_data[['game_id', 'player1', 'player2']]
val_other_info = val_data[['game_id', 'player1', 'player2']]
test_other_info = test_data[['game_id', 'player1', 'player2']]

In [7]:
train_x = train_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
train_y = train_data['Brownlow Votes']
val_x = val_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
val_y = val_data['Brownlow Votes']
test_x = test_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
test_y = test_data['Brownlow Votes']

In [8]:
# ningxiang.read_in_train_data(train_x, train_y)
# ningxiang.set_model_type("Regression")
# ningxiang_output = ningxiang.get_rf_based_feature_combinations(3)

# with open(f'./drive/My Drive/LAB/BrownlowH2H/models/NingXing_output.pickle', 'rb') as f:
#     ningxiang_output = pickle.load(f)

In [9]:
with open('./drive/My Drive/LAB/BrownlowH2H/models/H2H_NingXiang.pickle', 'rb') as f:
  ningxiang_output = pickle.load(f)

Set parameters

In [10]:
import GuangAnB_H2H

In [11]:
guangan = GuangAnB_H2H.GuangAnB()

GuangAn Initialised


In [12]:
from sklearn.ensemble import HistGradientBoostingRegressor as HGBR

In [13]:
guangan.read_in_model(HGBR, 'Regression')

Successfully read in model <class 'sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingRegressor'>, which is a Regression model


In [14]:
guangan.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data


In [15]:
parameter_choices = {
    'max_depth': set(range(3, 193)),
    'max_bins': set(range(4, 256)),
    'interaction_cst': {'no_interactions', 'pairwise'},
    'learning_rate': (-8, -1),
    'l2_regularization': (-4, 1)

}

In [16]:
guangan.set_hyperparameters(parameter_choices)

Successfully recorded hyperparameter choices


In [17]:
guangan.read_in_transform({'learning_rate': '10^', 'l2_regularization': '10^'})

Updated transform dictionary: {'max_depth': False, 'max_bins': False, 'interaction_cst': False, 'learning_rate': '10^', 'l2_regularization': '10^'}


In [18]:
guangan.read_in_categorical(['interaction_cst'])

Updated categorical dictionary: {'max_depth': False, 'max_bins': False, 'interaction_cst': True, 'learning_rate': False, 'l2_regularization': False}
Updated original bounds dict: [({3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 18

In [19]:
guangan.set_non_tuneable_hyperparameters({'random_state': 19260817})

Successfully recorded non_tuneable_hyperparameter choices


In [20]:
guangan.set_features(ningxiang_output)

Successfully recorded tuneable feature combination choices and updated relevant internal structures


In [21]:
try:
  guangan.read_in_tuning_result_df(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv', f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')
  print('True')
except:
  print('False')

Successfully read in tuning result of 1288 rows
True


In [22]:
guangan.set_tuning_result_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv')

Successfully set tuning output address


In [23]:
guangan.set_best_model_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tmp_models/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')

Successfully set best model output address


In [24]:
guangan.train_other_info = train_other_info
guangan.val_other_info = val_other_info
guangan.test_other_info = test_other_info

In [25]:
import warnings
warnings.filterwarnings("ignore")

In [26]:
guangan.tune()

Streaming output truncated to the last 5000 lines.
        Current best combo: (3, 4, 'pairwise', -1.875, -4.0, 27) with val score 0.5912. 
        Has trained 1288 combinations so far
Already Trained and Tested combination {'max_depth': 27, 'max_bins': 36, 'interaction_cst': 'pairwise', 'learning_rate': -1.875, 'l2_regularization': -3.375, 'features': 31}, which had val score of 0.586
        Current best combo: (3, 4, 'pairwise', -1.875, -4.0, 27) with val score 0.5912. 
        Has trained 1288 combinations so far
Already Trained and Tested combination {'max_depth': 27, 'max_bins': 36, 'interaction_cst': 'pairwise', 'learning_rate': -1.875, 'l2_regularization': -3.375, 'features': 35}, which had val score of 0.586
        Current best combo: (3, 4, 'pairwise', -1.875, -4.0, 27) with val score 0.5912. 
        Has trained 1288 combinations so far
Already Trained and Tested combination {'max_depth': 27, 'max_bins': 36, 'interaction_cst': 'pairwise', 'learning_rate': -1.0, 'l2_regulari

In [27]:
guangan.view_best_combo_and_score()

Max Score: 
 0.591194968553459
Max Test Score: 
 0.591195
Max Combo Index: 
 (3, 4, 'pairwise', -1.875, -4.0, 27)
Max Combo Hyperparamer Combination: 
 {'max_depth': 3, 'max_bins': 4, 'interaction_cst': 'pairwise', 'learning_rate': -1.875, 'l2_regularization': -4.0, 'features': 27}
Max Combo Features: 
 ('AFLCA_votes', 'Disposals', 'Contested Possessions', 'Goals', 'Centre Clearances', 'Intercepts', 'Clearances', 'Marks Inside 50', 'Hitouts', 'Handballs', 'Behinds', 'Contested Marks', 'Bounces', 'Ineffective Disposals', 'Stoppage Clearances', 'Tackles Inside 50', 'Rebound 50s', 'One Percenters', 'Goal Assists', 'Uncontested Possessions', 'Frees Agains', 'Inside 50s', 'Frees For', 'Metres Gained', 'Score Involvements', 'Behind Assists', 'Tackles Outside 50', 'Tackles', 'Clangers', 'Uncontested Marks')
# Combos Checked: 1388
